In [ ]:
import os
import shutil

# Source folder with all images
source_folder = '/kaggle/input/water-data/images'

# Destination folder where we’ll store only the lake images
destination_folder = '/kaggle/working/harbor_data'

# Create destination folder if it doesn't exist
os.makedirs(destination_folder, exist_ok=True)

# Loop through all files in the source folder
for filename in os.listdir(source_folder):
    if filename.lower().startswith('harbor') and filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        source_path = os.path.join(source_folder, filename)
        destination_path = os.path.join(destination_folder, filename)
        shutil.copy(source_path, destination_path)

print(f"✅ All lake images have been copied to: {destination_folder}")


In [ ]:
import numpy as np
import cv2
import os

# Folder with lake images
lake_folder = '/kaggle/working/harbor_data'

# Target image size
img_size = (128, 128)

# Lists to hold the images
lake_images_gray = []

# Loop over all lake images
for filename in os.listdir(lake_folder):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(lake_folder, filename)
        
        # Load image in grayscale
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        # Resize to uniform shape
        img = cv2.resize(img, img_size)
        
        # Normalize pixel values to range [0, 1]
        img = img / 255.0
        
        # Add to list
        lake_images_gray.append(img)

# Convert to NumPy array and expand dims to add channel (needed for CNN/U-Net)
X_gray = np.array(lake_images_gray)
X_gray = np.expand_dims(X_gray, axis=-1)  # shape: (num_images, 128, 128, 1)

print("✅ Preprocessing complete!")
print("Shape of preprocessed data:", X_gray.shape)


In [ ]:
import os
import numpy as np
import cv2

# Paths
lake_images_folder = '/kaggle/working/harbor_data'
generated_masks_folder = '/kaggle/working/generated_masks'
img_size = (128, 128)

# Create a folder to save generated masks
os.makedirs(generated_masks_folder, exist_ok=True)

# Loop through all lake images
for filename in os.listdir(lake_images_folder):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(lake_images_folder, filename)
        
        # Load the image in grayscale
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        # Resize to uniform size
        img = cv2.resize(img, img_size)
        
        # Thresholding: Water is assumed to have low intensity (darker pixels)
        # We use a threshold value (e.g., 100), but this may need adjustment based on the dataset
        _, mask = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY)
        
        # Normalize the mask to be between 0 and 1 (for consistency with model input)
        mask = mask / 255.0
        
        # Save the generated mask
        mask_path = os.path.join(generated_masks_folder, filename)
        cv2.imwrite(mask_path, (mask * 255).astype(np.uint8))  # save as 0 or 255 mask image

print("✅ Masks have been generated and saved to:", generated_masks_folder)


In [ ]:
import numpy as np
import cv2
import os

# Folder with the lake images (preprocessed) and corresponding masks
lake_folder = '/kaggle/working/harbor_data'
mask_folder = '/kaggle/working/generated_masks'

# Target image size
img_size = (128, 128)

# Lists to hold the processed images
lake_images_bw = []
lake_images_intensity = []

# Function to classify intensity
def classify_intensity(gray_image):
    # Define intensity thresholds for low, medium, and high
    low_threshold = 0.33
    medium_threshold = 0.66
    
    # Normalize the image to range [0, 1]
    normalized_image = gray_image / 255.0
    
    # Classify each pixel intensity into low (0), medium (1), or high (2)
    intensity_map = np.digitize(normalized_image, bins=[low_threshold, medium_threshold])
    
    return intensity_map

# Loop over all lake images
for filename in os.listdir(lake_folder):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(lake_folder, filename)
        
        # Load the image in grayscale (black and white)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        # Resize to uniform shape
        img = cv2.resize(img, img_size)
        
        # Load corresponding mask (assuming masks are named the same as images)
        mask_path = os.path.join(mask_folder, filename)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, img_size)  # Resize mask to match image dimensions
        
        # Normalize the mask (optional, depending on mask format)
        mask = mask / 255.0  # Mask values will be 0 (land) or 1 (water)
        
        # Apply mask: Set land areas (where mask is 0) to 0 in the image
        img_masked = img * mask
        
        # Normalize the masked image to range [0, 1]
        img_masked = img_masked / 255.0
        
        # Add the processed grayscale image (water areas only) to list
        lake_images_bw.append(img_masked)
        
        # Classify the intensity for the water areas
        intensity_map = classify_intensity(img_masked)
        
        # Add the intensity map to the list
        lake_images_intensity.append(intensity_map)

# Convert to NumPy arrays
X_bw = np.array(lake_images_bw)  # shape: (num_images, 128, 128)
X_intensity = np.array(lake_images_intensity)  # shape: (num_images, 128, 128)

# Expanding dimensions to add the channel for CNN/U-Net (if needed)
X_bw = np.expand_dims(X_bw, axis=-1)  # shape: (num_images, 128, 128, 1)
X_intensity = np.expand_dims(X_intensity, axis=-1)  # shape: (num_images, 128, 128, 1)

# Print output shapes
print("✅ Preprocessing complete!")
print("Shape of grayscale images:", X_bw.shape)
print("Shape of intensity mapped images:", X_intensity.shape)

# Optional: Visualize some of the images to confirm
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
for i in range(5):
    plt.subplot(1, 5, i+1)
    plt.imshow(X_bw[i].reshape(img_size), cmap='gray')
    plt.title(f"Intensity Map: {np.unique(X_intensity[i])}")
    plt.axis('off')
plt.show()


In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

# Folder with lake images
lake_folder = '/kaggle/working/harbor_data'

# Folder with masks
mask_folder = '/kaggle/working/generated_masks'

# Target image size
img_size = (128, 128)

# Function to load images and preprocess
def load_images_and_masks(lake_folder, mask_folder, img_size):
    images = []
    masks = []
    
    for filename in os.listdir(lake_folder):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(lake_folder, filename)
            mask_path = os.path.join(mask_folder, filename)

            # Load the image
            img = cv2.imread(img_path)
            img = cv2.resize(img, img_size)  # Resize to uniform shape
            images.append(img)

            # Load the mask
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Read the mask in grayscale
            mask = cv2.resize(mask, img_size)  # Resize to uniform shape
            masks.append(mask)
    
    return np.array(images), np.array(masks)

# Load the images and masks
images, masks = load_images_and_masks(lake_folder, mask_folder, img_size)

# Function to calculate intensity based on the blue channel and classify depth
def calculate_depth_intensity(image, mask):
    # Apply the mask to the image (select only water regions)
    water_region = cv2.bitwise_and(image, image, mask=mask)
    
    # Extract the blue channel (index 0 in BGR images)
    blue_channel = water_region[:, :, 0]

    # Calculate the average intensity of the blue channel for the water region
    blue_intensity = np.mean(blue_channel)  # Average across the height and width of the image

    # Normalize the blue intensity (between 0 and 255)
    blue_intensity_normalized = blue_intensity / 255.0

    # Classify depth as Low, Medium, or High based on blue intensity
    # Here, darker blue (lower values) correspond to higher depth
    if blue_intensity_normalized < 0.33:
        return 'High Depth', blue_intensity
    elif blue_intensity_normalized < 0.66:
        return 'Medium Depth', blue_intensity
    else:
        return 'Low Depth', blue_intensity

# Loop through all images and calculate the depth intensity
depth_labels = []
depth_values = []

for img, mask in zip(images, masks):
    label, depth = calculate_depth_intensity(img, mask)
    depth_labels.append(label)
    depth_values.append(depth)

# Display some of the results
for i in range(5):  # Show 5 random results
    plt.imshow(images[i])
    plt.title(f"Depth: {depth_labels[i]} - Blue Intensity: {depth_values[i]}")
    plt.axis('off')
    plt.show()

print("✅ Depth calculation and classification completed!")


In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Function to load images and masks
def load_images_and_masks(lake_folder, mask_folder, img_size):
    images = []
    masks = []
    
    for filename in os.listdir(lake_folder):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(lake_folder, filename)
            mask_path = os.path.join(mask_folder, filename)

            img = cv2.imread(img_path)
            img = cv2.resize(img, img_size)
            images.append(img)

            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, img_size)
            masks.append(mask)
    
    return np.array(images), np.array(masks)

# Paths
lake_folder = '/kaggle/working/harbor_data'
mask_folder = '/kaggle/working/generated_masks'
img_size = (128, 128)

# Load data
images, masks = load_images_and_masks(lake_folder, mask_folder, img_size)

# Depth calculation
def calculate_depth_intensity(image, mask):
    water_region = cv2.bitwise_and(image, image, mask=mask)
    blue_channel = water_region[:, :, 0]
    blue_intensity = np.mean(blue_channel)
    blue_intensity_normalized = blue_intensity / 255.0

    if blue_intensity_normalized < 0.33:
        return 'High Depth', blue_intensity
    elif blue_intensity_normalized < 0.66:
        return 'Medium Depth', blue_intensity
    else:
        return 'Low Depth', blue_intensity

# Label preparation
depth_labels = []
for img, mask in zip(images, masks):
    label, _ = calculate_depth_intensity(img, mask)
    depth_labels.append(label)

label_map = {'Low Depth': 0, 'Medium Depth': 1, 'High Depth': 2}
y = np.array([label_map[label] for label in depth_labels])

# Introduce small noise: only 3% wrong labels
np.random.seed(42)
num_wrong_labels = int(len(y) * 0.03)  # 3% mislabel
wrong_indices = np.random.choice(len(y), num_wrong_labels, replace=False)
for idx in wrong_indices:
    y[idx] = np.random.choice([0, 1, 2])

# Encode labels
y_categorical = to_categorical(y, num_classes=3)

# Normalize images
X = images / 255.0

# Train/test split
X_train, X_val, y_train, y_val = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Model with slight Dropout
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3), kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.2),  # One small dropout here

    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.25),  # One more small dropout here
    Dense(3, activation='softmax')
])

# Compile
optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=16,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]
)

# Plot
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Save
model.save('Corrected_CNN_LAKE.h5')
print("✅ Model training done!")

# ➡️ Final Accuracy Printout
final_train_acc = history.history['accuracy'][-1] * 100
final_val_acc = history.history['val_accuracy'][-1] * 100

print(f"🎯 Final Training Accuracy: {final_train_acc:.2f}%")
print(f"🎯 Final Validation Accuracy: {final_val_acc:.2f}%")
print("✅ Model saved as 'Corrected_CNN_LAKE.h5'")

# After training your model
model.save('harbor_prediction.keras')


In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, Add, Cropping2D
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

# Load images and masks as before
def load_images_and_masks(lake_folder, mask_folder, img_size):
    images = []
    masks = []
    for filename in os.listdir(lake_folder):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(lake_folder, filename)
            mask_path = os.path.join(mask_folder, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, img_size)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, img_size)
            images.append(img)
            masks.append(mask)
    return np.array(images), np.array(masks)

# Setup
lake_folder = '/kaggle/working/harbor_data'
mask_folder = '/kaggle/working/generated_masks'
img_size = (128, 128)
images, masks = load_images_and_masks(lake_folder, mask_folder, img_size)

# Normalize
X = images / 255.0
masks = masks / 255.0
masks = masks[..., np.newaxis]  # Add channel dimension

# Split
X_train, X_val, y_train, y_val = train_test_split(X, masks, test_size=0.2, random_state=42)

# Build FCN model
def build_fcn_model(input_shape):
    inputs = Input(shape=input_shape)
    
    # Encoder
    x1 = Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
    p1 = MaxPooling2D((2,2))(x1)
    
    x2 = Conv2D(64, (3,3), activation='relu', padding='same')(p1)
    p2 = MaxPooling2D((2,2))(x2)

    x3 = Conv2D(128, (3,3), activation='relu', padding='same')(p2)

    # Decoder
    u1 = Conv2DTranspose(64, (2,2), strides=(2,2), padding='same')(x3)
    m1 = Add()([u1, x2])
    
    u2 = Conv2DTranspose(32, (2,2), strides=(2,2), padding='same')(m1)
    m2 = Add()([u2, x1])

    outputs = Conv2D(1, (1,1), activation='sigmoid', padding='same')(m2)

    model = Model(inputs, outputs)
    return model

model = build_fcn_model((128, 128, 3))
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=16)

# Save model
model.save('FCN_LAKE_SEGMENTATION.h5')


# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Show final validation accuracy
final_val_acc = history.history['val_accuracy'][-1] * 100
print(f"✅ Model training complete. Final Validation Accuracy: {final_val_acc:.2f}%")
print("✅ Model saved as 'FCN_LAKE_SEGMENTATION.h5'")


In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Load images and masks
def load_images_and_masks(lake_folder, mask_folder, img_size):
    images = []
    masks = []
    for filename in os.listdir(lake_folder):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(lake_folder, filename)
            mask_path = os.path.join(mask_folder, filename)

            img = cv2.imread(img_path)
            img = cv2.resize(img, img_size)

            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, img_size)

            images.append(img)
            masks.append(mask)
    return np.array(images), np.array(masks)

lake_folder = '/kaggle/working/harbor_data'
mask_folder = '/kaggle/working/generated_masks'
img_size = (128, 128)

images, masks = load_images_and_masks(lake_folder, mask_folder, img_size)

# Preprocess images: apply mask to keep water regions only
def apply_mask(img, mask):
    return cv2.bitwise_and(img, img, mask=mask)

masked_images = [apply_mask(img, msk) for img, msk in zip(images, masks)]

# Calculate depth label from average blue intensity
def calculate_depth_label(image):
    blue_channel = image[:, :, 0]
    mean_blue = np.mean(blue_channel)
    norm = mean_blue / 255.0
    if norm < 0.33:
        return 2  # High Depth
    elif norm < 0.66:
        return 1  # Medium Depth
    else:
        return 0  # Low Depth

depth_labels = [calculate_depth_label(img) for img in masked_images]
y = to_categorical(depth_labels, num_classes=3)

X = np.array(masked_images) / 255.0

# Train/test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Load ResNet + GAP
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze base layers
for layer in base_model.layers:
    layer.trainable = False

# Add classification head
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Train
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=16)

# Plot accuracy
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('ResNet + GAP Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Save model
model.save('RESNET_GAP_LAKE.h5')
print("✅ Model saved as 'RESNET_GAP_LAKE.h5'")
final_val_acc = history.history['val_accuracy'][-1] * 100
print(f"✅ Model training complete. Final Validation Accuracy: {final_val_acc:.2f}%")
